# Predicting House Sale Prices with Linear Regression

In this project, I will be cleaning a dataset and transforming features to be used for training a machine learning model to predict house sale prices.

In [1]:
# Import necessary classes and modules
import pandas as pd
import numpy as np

from sklearn.model_selection import KFold
from sklearn.metrics import mean_squared_error
from sklearn import linear_model

import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
# Read in data set
df = pd.read_csv("AmesHousing.tsv", delimiter="\t")

In [27]:
# Create a few functions to help run experiments easily
def transform_features(df):
    return df

def select_features(df):
    return df[["Gr Liv Area","SalePrice"]]
    
def train_and_test(df):
    train = df[:1460]
    test = df[1460:]
    
    # Select only numerical columns
    numeric_train = train.select_dtypes(include = ['float', 'integer'])
    numeric_test = test.select_dtypes(include = ['float', 'integer'])
    
    # Remove 'SalePrice' (target column) from numerical columns
    features = numeric_train.columns.drop('SalePrice')
    # Create a linear regression instance and train model
    lr = linear_model.LinearRegression()
    lr.fit(train[features], train['SalePrice'])
    # Use model to generate predictions
    predictions = lr.predict(test[features])
    # Use RMSE as metric for our model
    mse = mean_squared_error(test['SalePrice'], predictions)
    rmse = np.sqrt(mse)
    
    return rmse

transform_df = transform_features(df)
filtered_df = select_features(transform_df)
rmse = train_and_test(filtered_df)

rmse

57088.25161263909

Now that we've defined a model, we can begin removing features that have too many missing values and transform text and numerical columns into the proper format, and potentially engineer new features.

In [8]:
# Drop columns containing more than 5% missing values
num_missing = df.isnull().sum() # Series of columns and count of missing values
drop_cols = num_missing[(num_missing > len(df)/20)].sort_values() # Datafram of columns to drop
df = df.drop(drop_cols.index, axis = 1)

In [9]:
# Drop text columns with more than 1 missing value for now
text_mv_cols = df.select_dtypes(include = ['object']).isnull().sum().sort_values(ascending = False)
drop_text_cols = text_mv_cols[(text_mv_cols > 0)]
df = df.drop(drop_text_cols.index, axis = 1)

In [15]:
# Compute missing values for numerical columns
num_missing = df.select_dtypes(include = ['float', 'integer']).isnull().sum()
fixable_num_missing = num_missing[(num_missing >0) & (num_missing < len(df)/20)]
fixable_num_missing

Mas Vnr Area      23
BsmtFin SF 1       1
BsmtFin SF 2       1
Bsmt Unf SF        1
Total Bsmt SF      1
Bsmt Full Bath     2
Bsmt Half Bath     2
Garage Cars        1
Garage Area        1
dtype: int64

In [16]:
# Compute the most common value from each fixable column
replacement_values_dict = df[fixable_num_missing.index].mode().to_dict(orient = 'records')[0]
replacement_values_dict

{'Bsmt Full Bath': 0.0,
 'Bsmt Half Bath': 0.0,
 'Bsmt Unf SF': 0.0,
 'BsmtFin SF 1': 0.0,
 'BsmtFin SF 2': 0.0,
 'Garage Area': 0.0,
 'Garage Cars': 2.0,
 'Mas Vnr Area': 0.0,
 'Total Bsmt SF': 0.0}

In [17]:
# Replace missing values with the most common value
df = df.fillna(replacement_values_dict)

In [18]:
# Verify that the data set has no missing values
df.isnull().sum()

Order              0
PID                0
MS SubClass        0
MS Zoning          0
Lot Area           0
Street             0
Lot Shape          0
Land Contour       0
Utilities          0
Lot Config         0
Land Slope         0
Neighborhood       0
Condition 1        0
Condition 2        0
Bldg Type          0
House Style        0
Overall Qual       0
Overall Cond       0
Year Built         0
Year Remod/Add     0
Roof Style         0
Roof Matl          0
Exterior 1st       0
Exterior 2nd       0
Mas Vnr Area       0
Exter Qual         0
Exter Cond         0
Foundation         0
BsmtFin SF 1       0
BsmtFin SF 2       0
                  ..
Central Air        0
1st Flr SF         0
2nd Flr SF         0
Low Qual Fin SF    0
Gr Liv Area        0
Bsmt Full Bath     0
Bsmt Half Bath     0
Full Bath          0
Half Bath          0
Bedroom AbvGr      0
Kitchen AbvGr      0
Kitchen Qual       0
TotRms AbvGrd      0
Functional         0
Fireplaces         0
Garage Cars        0
Garage Area  

Can we engineer any more features to capture information?

In [20]:
# Create a new column for years since remodeled. Note to remove rows that have negative data.
# Calculate difference between years of house built and house sold
years_sold = df['Yr Sold'] - df['Year Built']
years_sold = years_sold[years_sold < 0]
years_sold

2180   -1
dtype: int64

In [21]:
# Calculate difference between years of house built and house remodeled
years_remodeled = df['Yr Sold'] - df['Year Remod/Add']
years_remodeled = years_remodeled[years_remodeled < 0]
years_remodeled

1702   -1
2180   -2
2181   -1
dtype: int64

In [23]:
# Create new columns
df['Years Before Sale'] = years_sold
df['Years Since Remodel'] = years_remodeled

# Drop rows with negative values for both of these features
df.drop([1702, 2180, 2181], axis = 0)

# Drop original year columns
df.drop(['Yr Sold', 'Year Remod/Add'], axis = 1)

,Order,PID,MS SubClass,MS Zoning,Lot Area,Street,Lot Shape,Land Contour,Utilities,Lot Config,...,Pool Area,Misc Val,Mo Sold,Sale Type,Sale Condition,SalePrice,years_until_sold,years_until_remod,Years Before Sale,Years Since Remodel
0,1,526301100,20,RL,31770,Pave,IR1,Lvl,AllPub,Corner,...,0,0,5,WD,Normal,215000,NaN,NaN,NaN,NaN
1,2,526350040,20,RH,11622,Pave,Reg,Lvl,AllPub,Inside,...,0,0,6,WD,Normal,105000,NaN,NaN,NaN,NaN
2,3,526351010,20,RL,14267,Pave,IR1,Lvl,AllPub,Corner,...,0,12500,6,WD,Normal,172000,NaN,NaN,NaN,NaN
3,4,526353030,20,RL,11160,Pave,Reg,Lvl,AllPub,Corner,...,0,0,4,WD,Normal,244000,NaN,NaN,NaN,NaN
4,5,527105010,60,RL,13830,Pave,IR1,Lvl,AllPub,Inside,...,0,0,3,WD,Normal,189900,NaN,NaN,NaN,NaN
5,6,527105030,60,RL,9978,Pave,IR1,Lvl,AllPub,Inside,...,0,0,6,WD,Normal,195500,NaN,NaN,NaN,NaN
6,7,527127150,120,RL,4920,Pave,Reg,Lvl,AllPub,Inside,...,0,0,4,WD,Normal,213500,NaN,NaN,NaN,NaN
7,8,527145080,120,RL,5005,Pave,IR1,HLS,AllPub,Inside,...,0,0,1,WD,Normal,191500,NaN,NaN,NaN,NaN
8,9,527146030,120,RL,5389,Pave,IR1,Lvl,AllPub,Inside,...,0,0,3,WD,Normal,236500,NaN,NaN,NaN,NaN
9,10,527162130,60,RL,7500,Pave,Reg,Lvl,AllPub,Inside,...,0,0,6,WD,Normal,189000,NaN,NaN,NaN,NaN


In [24]:
# Drop columns that aren't useful for ML like identifiers
df = df.drop(["PID", "Order"], axis=1)

In [25]:
# Drop columns that leak data about the final sale
df = df.drop(["Mo Sold", "Sale Condition", "Sale Type", "Yr Sold"], axis=1)

We'll update our transform_features function with these updates.

In [30]:
# Update transform_features function to clean columns
def transform_features(df):
    num_missing = df.isnull().sum() # Series of columns and count of missing values
    drop_cols = num_missing[(num_missing > len(df)/20)].sort_values() # Datafram of columns to drop
    df = df.drop(drop_cols.index, axis = 1)

    text_mv_cols = df.select_dtypes(include = ['object']).isnull().sum().sort_values(ascending = False)
    drop_text_cols = text_mv_cols[(text_mv_cols > 0)]
    df = df.drop(drop_text_cols.index, axis = 1)
    
    num_missing = df.select_dtypes(include = ['float', 'integer']).isnull().sum()
    fixable_num_missing = num_missing[(num_missing >0) & (num_missing < len(df)/20)]
    replacement_values_dict = df[fixable_num_missing.index].mode().to_dict(orient = 'records')[0]
    df = df.fillna(replacement_values_dict)

    years_sold = df['Yr Sold'] - df['Year Built']  
    years_remodeled = df['Yr Sold'] - df['Year Remod/Add'] 
    df['Years Before Sale'] = years_sold
    df['Years Since Remodel'] = years_remodeled
    
    df = df.drop([1702, 2180, 2181], axis = 0)
  
    # Drop columns that aren't useful for ML like identifiers
    df = df.drop(["PID", "Order"], axis=1)
    
    df = df.drop(["Mo Sold", "Sale Condition", "Sale Type", "Yr Sold", "Year Built", "Year Remod/Add"], axis=1)

    return df

def select_features(df):
    return df[["Gr Liv Area","SalePrice"]]
    
def train_and_test(df):
    train = df[:1460]
    test = df[1460:]
    
    # Select only numerical columns
    numeric_train = train.select_dtypes(include = ['float', 'integer'])
    numeric_test = test.select_dtypes(include = ['float', 'integer'])
    
    # Remove 'SalePrice' (target column) from numerical columns
    features = numeric_train.columns.drop('SalePrice')
    # Create a linear regression instance and train model
    lr = linear_model.LinearRegression()
    lr.fit(train[features], train['SalePrice'])
    # Use model to generate predictions
    predictions = lr.predict(test[features])
    # Use RMSE as metric for our model
    mse = mean_squared_error(test['SalePrice'], predictions)
    rmse = np.sqrt(mse)
    
    return rmse

df = pd.read_csv("AmesHousing.tsv", delimiter="\t")
transform_df = transform_features(df)
filtered_df = select_features(transform_df)
rmse = train_and_test(filtered_df)

rmse

55275.36731241307

Our model currently uses all features that are numerical with no missing values. We will generate a correlation heatmap matrix of the numerical features in the training data set to find the features that correlate well with SalePrice.

In [32]:
numerical_df = transform_df.select_dtypes(include = ['float', 'integer'])
numerical_df

,MS SubClass,Lot Area,Overall Qual,Overall Cond,Mas Vnr Area,BsmtFin SF 1,BsmtFin SF 2,Bsmt Unf SF,Total Bsmt SF,1st Flr SF,...,Wood Deck SF,Open Porch SF,Enclosed Porch,3Ssn Porch,Screen Porch,Pool Area,Misc Val,SalePrice,Years Before Sale,Years Since Remodel
0,20,31770,6,5,112.0,639.0,0.0,441.0,1080.0,1656,...,210,62,0,0,0,0,0,215000,50,50
1,20,11622,5,6,0.0,468.0,144.0,270.0,882.0,896,...,140,0,0,0,120,0,0,105000,49,49
2,20,14267,6,6,108.0,923.0,0.0,406.0,1329.0,1329,...,393,36,0,0,0,0,12500,172000,52,52
3,20,11160,7,5,0.0,1065.0,0.0,1045.0,2110.0,2110,...,0,0,0,0,0,0,0,244000,42,42
4,60,13830,5,5,0.0,791.0,0.0,137.0,928.0,928,...,212,34,0,0,0,0,0,189900,13,12
5,60,9978,6,6,20.0,602.0,0.0,324.0,926.0,926,...,360,36,0,0,0,0,0,195500,12,12
6,120,4920,8,5,0.0,616.0,0.0,722.0,1338.0,1338,...,0,0,170,0,0,0,0,213500,9,9
7,120,5005,8,5,0.0,263.0,0.0,1017.0,1280.0,1280,...,0,82,0,0,144,0,0,191500,18,18
8,120,5389,8,5,0.0,1180.0,0.0,415.0,1595.0,1616,...,237,152,0,0,0,0,0,236500,15,14
9,60,7500,7,5,0.0,0.0,0.0,994.0,994.0,1028,...,140,60,0,0,0,0,0,189000,11,11


In [33]:
numerical_df.shape

(2927, 33)

In [35]:
abs_corr_coeffs = numerical_df.corr()['SalePrice'].abs().sort_values(ascending = False)
abs_corr_coeffs

SalePrice              1.000000
Overall Qual           0.801206
Gr Liv Area            0.717596
Garage Cars            0.648361
Total Bsmt SF          0.644012
Garage Area            0.641425
1st Flr SF             0.635185
Years Before Sale      0.558979
Full Bath              0.546118
Years Since Remodel    0.534985
Mas Vnr Area           0.506983
TotRms AbvGrd          0.498574
Fireplaces             0.474831
BsmtFin SF 1           0.439284
Wood Deck SF           0.328183
Open Porch SF          0.316262
Half Bath              0.284871
Bsmt Full Bath         0.276258
2nd Flr SF             0.269601
Lot Area               0.267520
Bsmt Unf SF            0.182751
Bedroom AbvGr          0.143916
Enclosed Porch         0.128685
Kitchen AbvGr          0.119760
Screen Porch           0.112280
Overall Cond           0.101540
MS SubClass            0.085128
Pool Area              0.068438
Low Qual Fin SF        0.037629
Bsmt Half Bath         0.035875
3Ssn Porch             0.032268
Misc Val

Overall Qual, Gr Liv Area, Garage Cars, Total Bsmt SF, and Garage Area correlate well with SalePrice. We'll use a threshold of correlation coeffecient of 0.6 to begin with and use the columns that correlate above that threshold as features for our model and test if it results in a better cross validation score.

In [38]:
# Keep columns with a correlation coefficient of larger than 0.6 (arbitrary- will experiment later!)
abs_corr_coeffs[abs_corr_coeffs > 0.6]

SalePrice        1.000000
Overall Qual     0.801206
Gr Liv Area      0.717596
Garage Cars      0.648361
Total Bsmt SF    0.644012
Garage Area      0.641425
1st Flr SF       0.635185
Name: SalePrice, dtype: float64

In [39]:
# Drop columns with less than 0.6 correlation with SalePrice
transform_df = transform_df.drop(abs_corr_coeffs[abs_corr_coeffs < 0.6].index, axis=1)

Next, which categorical columns should we keep as features? This would only be helpful if the column has a few unique values and captures variability in the data.

In [40]:
# Create a list of column names from documentation that are *meant* to be categorical
nominal_features = ["PID", "MS SubClass", "MS Zoning", "Street", "Alley", "Land Contour", "Lot Config", "Neighborhood", 
                    "Condition 1", "Condition 2", "Bldg Type", "House Style", "Roof Style", "Roof Matl", "Exterior 1st", 
                    "Exterior 2nd", "Mas Vnr Type", "Foundation", "Heating", "Central Air", "Garage Type", 
                    "Misc Feature", "Sale Type", "Sale Condition"]

In [41]:
# Explore which categorical columns we have carried with us in transform function
transform_cat_cols = []
for col in nominal_features:
    if col in transform_df.columns:
        transform_cat_cols.append(col)
transform_cat_cols

['MS Zoning',
 'Street',
 'Land Contour',
 'Lot Config',
 'Neighborhood',
 'Condition 1',
 'Condition 2',
 'Bldg Type',
 'House Style',
 'Roof Style',
 'Roof Matl',
 'Exterior 1st',
 'Exterior 2nd',
 'Foundation',
 'Heating',
 'Central Air']

In [42]:
# How many unique values in each categorical column?
uniqueness_counts = transform_df[transform_cat_cols].apply(lambda col: len(col.value_counts())).sort_values()
uniqueness_counts

Street           2
Central Air      2
Land Contour     4
Lot Config       5
Bldg Type        5
Roof Style       6
Foundation       6
Heating          6
MS Zoning        7
Condition 2      8
House Style      8
Roof Matl        8
Condition 1      9
Exterior 1st    16
Exterior 2nd    17
Neighborhood    28
dtype: int64

Since we will be generating a dummy column for each category, columns such as Neighborhood, Exterior 1st/2nd don't make sense with so many unique values. I'll use a threshold of 10 to cut off the feature selection of these text columns.

In [51]:
drop_nonuniq_cols = uniqueness_counts[uniqueness_counts > 10].index
transform_df = transform_df.drop(drop_nonuniq_cols, axis=1)

In [52]:
transform_df.shape

(2927, 29)

In [53]:
# Select the remaining text columns and convert to categorical
text_cols = transform_df.select_dtypes(include=['object'])
for col in text_cols:
    transform_df[col] = transform_df[col].astype('category')

In [54]:
# Create dummy columns and add back to the dataframe!
transform_df = pd.concat([
    transform_df, 
    pd.get_dummies(transform_df.select_dtypes(include=['category']))], axis=1)

In [55]:
transform_df.shape

(2927, 145)

Now that we have some selected features, we'll update our select_feature function to reflect that.

In [56]:
# Update transform_features function to clean columns
def transform_features(df):
    num_missing = df.isnull().sum() # Series of columns and count of missing values
    drop_cols = num_missing[(num_missing > len(df)/20)].sort_values() # Datafram of columns to drop
    df = df.drop(drop_cols.index, axis = 1)

    text_mv_cols = df.select_dtypes(include = ['object']).isnull().sum().sort_values(ascending = False)
    drop_text_cols = text_mv_cols[(text_mv_cols > 0)]
    df = df.drop(drop_text_cols.index, axis = 1)
    
    num_missing = df.select_dtypes(include = ['float', 'integer']).isnull().sum()
    fixable_num_missing = num_missing[(num_missing >0) & (num_missing < len(df)/20)]
    replacement_values_dict = df[fixable_num_missing.index].mode().to_dict(orient = 'records')[0]
    df = df.fillna(replacement_values_dict)

    years_sold = df['Yr Sold'] - df['Year Built']  
    years_remodeled = df['Yr Sold'] - df['Year Remod/Add'] 
    df['Years Before Sale'] = years_sold
    df['Years Since Remodel'] = years_remodeled
    
    df = df.drop([1702, 2180, 2181], axis = 0)
  
    # Drop columns that aren't useful for ML like identifiers
    df = df.drop(["PID", "Order"], axis=1)
    
    df = df.drop(["Mo Sold", "Sale Condition", "Sale Type", "Yr Sold", "Year Built", "Year Remod/Add"], axis=1)

    return df

def select_features(df, coeff_threshold = 0.6, uniq_threshold = 10):
    numerical_df = df.select_dtypes(include = ['float', 'integer'])
    abs_corr_coeffs = numerical_df.corr()['SalePrice'].abs().sort_values(ascending = False)
    # Drop columns with less than 0.6 correlation with SalePrice
    df = df.drop(abs_corr_coeffs[abs_corr_coeffs < coeff_threshold].index, axis=1)
    # Create a list of column names from documentation that are *meant* to be categorical
    nominal_features = ["PID", "MS SubClass", "MS Zoning", "Street", "Alley", "Land Contour", "Lot Config", "Neighborhood", 
                    "Condition 1", "Condition 2", "Bldg Type", "House Style", "Roof Style", "Roof Matl", "Exterior 1st", 
                    "Exterior 2nd", "Mas Vnr Type", "Foundation", "Heating", "Central Air", "Garage Type", 
                    "Misc Feature", "Sale Type", "Sale Condition"]
    # Explore which categorical columns we have carried with us in transform function
    transform_cat_cols = []
    for col in nominal_features:
        if col in df.columns:
            transform_cat_cols.append(col)
    uniqueness_counts = df[transform_cat_cols].apply(lambda col: len(col.value_counts())).sort_values()
    drop_nonuniq_cols = uniqueness_counts[uniqueness_counts > uniq_threshold].index
    df = df.drop(drop_nonuniq_cols, axis=1)
    
    # Select the remaining text columns and convert to categorical
    text_cols = df.select_dtypes(include=['object'])
    for col in text_cols:
        df[col] = df[col].astype('category')
    df = pd.concat([df, pd.get_dummies(df.select_dtypes(include=['category']))], axis=1)
    return df
    
def train_and_test(df):
    train = df[:1460]
    test = df[1460:]
    
    # Select only numerical columns
    numeric_train = train.select_dtypes(include = ['float', 'integer'])
    numeric_test = test.select_dtypes(include = ['float', 'integer'])
    
    # Remove 'SalePrice' (target column) from numerical columns
    features = numeric_train.columns.drop('SalePrice')
    # Create a linear regression instance and train model
    lr = linear_model.LinearRegression()
    lr.fit(train[features], train['SalePrice'])
    # Use model to generate predictions
    predictions = lr.predict(test[features])
    # Use RMSE as metric for our model
    mse = mean_squared_error(test['SalePrice'], predictions)
    rmse = np.sqrt(mse)
    
    return rmse

df = pd.read_csv("AmesHousing.tsv", delimiter="\t")
transform_df = transform_features(df)
filtered_df = select_features(transform_df)
rmse = train_and_test(filtered_df)

rmse

34360.44494162579

Now that we have our selected features implemented into our function, we can iterate with different features easily. Next, we'll also implent k-fold cross validation to optimize our hyperparameters.

In [58]:
# Update transform_features function to clean columns
def transform_features(df):
    num_missing = df.isnull().sum() # Series of columns and count of missing values
    drop_cols = num_missing[(num_missing > len(df)/20)].sort_values() # Datafram of columns to drop
    df = df.drop(drop_cols.index, axis = 1)

    text_mv_cols = df.select_dtypes(include = ['object']).isnull().sum().sort_values(ascending = False)
    drop_text_cols = text_mv_cols[(text_mv_cols > 0)]
    df = df.drop(drop_text_cols.index, axis = 1)
    
    num_missing = df.select_dtypes(include = ['float', 'integer']).isnull().sum()
    fixable_num_missing = num_missing[(num_missing >0) & (num_missing < len(df)/20)]
    replacement_values_dict = df[fixable_num_missing.index].mode().to_dict(orient = 'records')[0]
    df = df.fillna(replacement_values_dict)

    years_sold = df['Yr Sold'] - df['Year Built']  
    years_remodeled = df['Yr Sold'] - df['Year Remod/Add'] 
    df['Years Before Sale'] = years_sold
    df['Years Since Remodel'] = years_remodeled
    
    df = df.drop([1702, 2180, 2181], axis = 0)
  
    # Drop columns that aren't useful for ML like identifiers
    df = df.drop(["PID", "Order"], axis=1)
    
    df = df.drop(["Mo Sold", "Sale Condition", "Sale Type", "Yr Sold", "Year Built", "Year Remod/Add"], axis=1)

    return df

def select_features(df, coeff_threshold = 0.4, uniq_threshold = 10):
    numerical_df = df.select_dtypes(include = ['float', 'integer'])
    abs_corr_coeffs = numerical_df.corr()['SalePrice'].abs().sort_values(ascending = False)
    df = df.drop(abs_corr_coeffs[abs_corr_coeffs < coeff_threshold].index, axis=1)
    # Create a list of column names from documentation that are *meant* to be categorical
    nominal_features = ["PID", "MS SubClass", "MS Zoning", "Street", "Alley", "Land Contour", "Lot Config", "Neighborhood", 
                    "Condition 1", "Condition 2", "Bldg Type", "House Style", "Roof Style", "Roof Matl", "Exterior 1st", 
                    "Exterior 2nd", "Mas Vnr Type", "Foundation", "Heating", "Central Air", "Garage Type", 
                    "Misc Feature", "Sale Type", "Sale Condition"]
    # Explore which categorical columns we have carried with us in transform function
    transform_cat_cols = []
    for col in nominal_features:
        if col in df.columns:
            transform_cat_cols.append(col)
    uniqueness_counts = df[transform_cat_cols].apply(lambda col: len(col.value_counts())).sort_values()
    drop_nonuniq_cols = uniqueness_counts[uniqueness_counts > uniq_threshold].index
    df = df.drop(drop_nonuniq_cols, axis=1)
    
    # Select the remaining text columns and convert to categorical
    text_cols = df.select_dtypes(include=['object'])
    for col in text_cols:
        df[col] = df[col].astype('category')
    df = pd.concat([df, pd.get_dummies(df.select_dtypes(include=['category']))], axis=1)
    return df
    
def train_and_test(df, k = 0):
    numeric_df = df.select_dtypes(include=['integer', 'float'])
    features = numeric_df.columns.drop("SalePrice")
    # Create a linear regression instance and train model
    lr = linear_model.LinearRegression()
    
    if k == 0:
        train = df[:1460]
        test = df[1460:]

        lr.fit(train[features], train['SalePrice'])
        # Use model to generate predictions
        predictions = lr.predict(test[features])
        # Use RMSE as metric for our model
        mse = mean_squared_error(test['SalePrice'], predictions)
        rmse = np.sqrt(mse)
    
    if k == 1:
        # Randomize *all* rows (frac=1) from `df` and return
        shuffled_df = df.sample(frac=1, )
        train = df[:1460]
        test = df[1460:]
        
        lr.fit(train[features], train["SalePrice"])
        predictions_one = lr.predict(test[features])
        mse_one = mean_squared_error(test["SalePrice"], predictions_one)
        rmse_one = np.sqrt(mse_one)
        
        lr.fit(test[features], test["SalePrice"])
        predictions_two = lr.predict(train[features])        
        mse_two = mean_squared_error(train["SalePrice"], predictions_two)
        rmse_two = np.sqrt(mse_two)
        
        avg_rmse = np.mean([rmse_one, rmse_two])
        print(rmse_one)
        print(rmse_two)
        return avg_rmse
    
    else:
        kf = KFold(n_splits = k, shuffle = True)
        rmse_values = []
        for train_index, test_index in kf.split(df):
            train = df.iloc[train_index]
            test = df.iloc[test_index]
            lr.fit(train[features], train["SalePrice"])
            predictions = lr.predict(test[features])
            mse = mean_squared_error(test["SalePrice"], predictions)
            rmse = np.sqrt(mse)
            rmse_values.append(rmse)
        print(rmse_values)
        avg_rmse = np.mean(rmse_values)
        return avg_rmse
        
    return rmse

df = pd.read_csv("AmesHousing.tsv", delimiter="\t")
transform_df = transform_features(df)
filtered_df = select_features(transform_df)
rmse = train_and_test(filtered_df, k=4)

rmse

[24551.53705195953, 28513.660996165298, 36208.15862727411, 28364.77382412681]


29409.532624881438

We've built a model that takes in several features of Ames Housing to predict the sale price of a house. Our current model does much better than it did when it was using just the Gr Living Area to predict sale price. We have also implemented k-fold cross-validation to prevent overfitting.

# Next Steps

I will continue to iterate on feature engineering and further improve how to pick the categorical columns.